In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from scipy.stats import gamma
import sys
sys.path.append("../build") 

from IPSModule import NoseHooverSystem, IPS_Simulator_NoseHoover  # Import Nose–Hoover components

# -----------------------------
# Simulation parameters
# -----------------------------
gamma_ = 1.0
temperature = 0.5
dt = 0.0005
rad = 30.0
n_particles = 30
epsilon = 1.0
sigma = 1.0

# -----------------------------
# Initialize particle positions within a circular region
# -----------------------------
def generate_random_particles(n_particles=30, box_radius=20.0):
    np.random.seed(None)
    while True:
        positions = np.random.uniform(-box_radius, box_radius, size=(n_particles, 2))
        if np.all(np.linalg.norm(positions, axis=1) <= box_radius):
            return positions

# -----------------------------
# Count the number of clusters using DBSCAN
# -----------------------------
def count_clusters(positions, eps=1.7, min_samples=3):
    if np.isnan(positions).any():
        return 0
    db = DBSCAN(eps=eps, min_samples=min_samples)
    labels = db.fit_predict(positions)
    return len(set(labels)) - (1 if -1 in labels else 0)

# -----------------------------
# Run a single simulation and return the step when 3 clusters first appear
# -----------------------------
def simulate_until_five_clusters(max_steps=1500000, check_interval=500, eps=1.7, min_samples=3):
    init_positions = generate_random_particles(n_particles)
    Q = 1.0
    eta = 0.0
    system = NoseHooverSystem(n_particles, temperature, Q, eta)

    for i in range(n_particles):
        for d in range(2):
            system.get_positions()[d][i] = init_positions[i][d]
            system.get_velocities()[d][i] = 0.0

    sim = IPS_Simulator_NoseHoover(system)
    sim.init(
        {"type": "LennardJones", "eps": epsilon, "sigma": sigma},
        {"type": "Radial", "rad": rad}
    )

    for step in range(max_steps):
        sim.integrate(dt)
        if step % check_interval == 0:
            pos = np.array(system.get_positions()).T
            n_clusters = count_clusters(pos, eps=eps, min_samples=min_samples)
            if n_clusters >= 3:
                return step
    return max_steps

# -----------------------------
# Run multiple simulations and collect the formation times
# -----------------------------
def repeat_simulations(n_runs=500):
    times = []
    for i in range(n_runs):
        print(f"Run {i+1}/{n_runs}...")
        t = simulate_until_five_clusters()
        times.append(t)
        print(f"  Formed 3 clusters at step {t}")
    return times

# -----------------------------
# Plot histogram of cluster formation times
# -----------------------------
def plot_formation_time_distribution(times, label="Nose-Hoover"):
    filtered = np.array([t for t in times if t < 1500000])  # Remove failed runs
    counts, bins, _ = plt.hist(filtered, bins=30, edgecolor='k', alpha=0.6, label='Observed')
    
    # Optional: fit gamma distribution and overlay
    # shape, loc, scale = gamma.fit(filtered, floc=0)
    # x = np.linspace(min(filtered), max(filtered), 500)
    # pdf = gamma.pdf(x, shape, loc=loc, scale=scale) * len(filtered) * (bins[1] - bins[0])
    # plt.plot(x, pdf, 'r-', lw=2, label=f'Gamma fit\nshape={shape:.2f}, scale={scale:.2f}')

    plt.xlabel("Steps to form 3 clusters")
    plt.ylabel("Frequency")
    plt.title(f"Gamma Fit to Cluster Formation Times ({label})")
    plt.legend()
    plt.show()
    # print(f"Average: {np.mean(filtered):.2f} steps | Gamma mean ≈ {shape * scale:.2f}")

# -----------------------------
# Main execution
# -----------------------------
times = repeat_simulations(n_runs=500)
plot_formation_time_distribution(times)


In [ ]:
np.save("times_nosehoover_5clusters.npy", np.array(times))
times = np.load("times_nosehoover_5clusters.npy")

Distribution fitting result (Nose–Hoover)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

def analyze_cluster_times(filename, max_steps=1500000):
    # --- Load data ---
    times = np.load(filename)
    
    # Filter out invalid values: NaN, inf, values >= max_steps, or <= 0 (invalid for gamma/lognorm)
    filtered = times[np.isfinite(times)]  # Remove NaN and inf
    filtered = filtered[(filtered > 0) & (filtered < max_steps)]

    print(f"Loaded {len(times)} samples, with {len(times) - len(filtered)} filtered out.")

    # --- Candidate distributions ---
    distributions = {
        # 'gamma': stats.gamma,
        # 'weibull': stats.weibull_min,
        'lognorm': stats.lognorm,
        # 'expon': stats.expon
    }

    x = np.linspace(min(filtered), max(filtered), 500)
    plt.hist(filtered, bins=100, density=True, alpha=0.6, edgecolor='k', label='Observed')

    best_fit = None
    best_ks = (None, 0)

    print("\nDistribution fitting result (Nose–Hoover)")
    print("-" * 50)

    for name, dist in distributions.items():
        try:
            if name in ['gamma', 'expon']:
                params = dist.fit(filtered, floc=0)
            else:
                params = dist.fit(filtered)

            ks_stat, pval = stats.kstest(filtered, dist.name, args=params)
            pdf = dist.pdf(x, *params)
            plt.plot(x, pdf, lw=2, label=f'{name} (p={pval:.3f})')

            print(f"{name:<10s} KS={ks_stat:.4f}, p={pval:.4f}, params={np.round(params, 3)}")

            if pval > best_ks[1]:
                best_fit = name
                best_ks = (ks_stat, pval)

        except Exception as e:
            print(f"{name:<10s} Fail to fit: {e}")

    # --- Visualization ---
    plt.xlabel("Steps to form 3 clusters")
    plt.ylabel("Probability Density")
    plt.title("Cluster Formation Time Distribution Fit (Nose–Hoover)")
    plt.legend()
    plt.grid(True)
    plt.show()

    print(f"\nBest fit distribution: {best_fit} (p = {best_ks[1]:.3f})")
    return best_fit


In [ ]:
analyze_cluster_times("times_nosehoover_5clusters.npy", max_steps=1500000)

Nose-Hoover-Langevin

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from scipy.stats import gamma
import sys
sys.path.append("../build") 

from IPSModule import NoseHooverLangevinSystem, IPS_Simulator_NoseHooverLangevin

# -----------------------------
# Simulation parameters
# -----------------------------
gamma_ = 1.0
temperature = 0.5
dt = 0.0005
rad = 30.0
n_particles = 30
epsilon = 1.0
sigma = 1.0
Q = 1.0
eta = 0.0

# -----------------------------
# Generate initial particle positions
# -----------------------------
def generate_random_particles(n_particles=30, box_radius=20.0):
    np.random.seed(None)
    while True:
        positions = np.random.uniform(-box_radius, box_radius, size=(n_particles, 2))
        if np.all(np.linalg.norm(positions, axis=1) <= box_radius):
            return positions

# -----------------------------
# Count clusters using DBSCAN
# -----------------------------
def count_clusters(positions, eps=1.7, min_samples=3):
    if np.isnan(positions).any():
        return 0
    db = DBSCAN(eps=eps, min_samples=min_samples)
    labels = db.fit_predict(positions)
    return len(set(labels)) - (1 if -1 in labels else 0)

# -----------------------------
# Run one simulation and return step when 3 clusters form
# -----------------------------
def simulate_until_five_clusters(max_steps=1500000, check_interval=500, eps=1.7, min_samples=3):
    init_positions = generate_random_particles(n_particles)
    system = NoseHooverLangevinSystem(n_particles, gamma_, temperature, Q, eta)

    for i in range(n_particles):
        for d in range(2):
            system.get_positions()[d][i] = init_positions[i][d]
            system.get_velocities()[d][i] = 0.0

    sim = IPS_Simulator_NoseHooverLangevin(system)
    sim.init(
        {"type": "LennardJones", "eps": epsilon, "sigma": sigma},
        {"type": "Radial", "rad": rad}
    )

    for step in range(max_steps):
        sim.integrate(dt)
        if step % check_interval == 0:
            pos = np.array(system.get_positions()).T
            n_clusters = count_clusters(pos, eps=eps, min_samples=min_samples)
            if n_clusters >= 3:
                return step
    return max_steps

# -----------------------------
# Run multiple simulations
# -----------------------------
def repeat_simulations(n_runs=500):
    times = []
    for i in range(n_runs):
        print(f"Run {i+1}/{n_runs}...")
        t = simulate_until_five_clusters()
        times.append(t)
        print(f"  Formed 3 clusters at step {t}")
    return times

# -----------------------------
# Plot histogram of formation times
# -----------------------------
def plot_formation_time_distribution(times, label="Nose–Hoover–Langevin"):
    filtered = np.array([t for t in times if t < 1500000])
    counts, bins, _ = plt.hist(filtered, bins=30, edgecolor='k', alpha=0.6, label='Observed')

    # Uncomment to fit gamma distribution
    # shape, loc, scale = gamma.fit(filtered, floc=0)
    # x = np.linspace(min(filtered), max(filtered), 500)
    # pdf = gamma.pdf(x, shape, loc=loc, scale=scale) * len(filtered) * (bins[1] - bins[0])
    # plt.plot(x, pdf, 'r-', lw=2, label=f'Gamma fit\nshape={shape:.2f}, scale={scale:.2f}')

    plt.xlabel("Steps to form 3 clusters")
    plt.ylabel("Frequency")
    plt.title(f"Cluster Formation Time Histogram ({label})")
    plt.legend()
    plt.show()

    print(f"Average steps: {np.mean(filtered):.2f} | Median: {np.median(filtered)}")

# -----------------------------
# Main execution
# -----------------------------
times = repeat_simulations(n_runs=500)
np.save("times_nosehoover_langevin.npy", np.array(times))
plot_formation_time_distribution(times)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

def analyze_cluster_times(filename, max_steps=1_500_000):
    times = np.load(filename)
    filtered = times[times < max_steps]

    print(f"Loaded {len(times)} samples, with {len(times) - len(filtered)} unclustered samples filtered out.")

    distributions = {
        'gamma': stats.gamma,
        # 'weibull': stats.weibull_min,
        # 'lognorm': stats.lognorm,
        # 'expon': stats.expon
    }

    x = np.linspace(min(filtered), max(filtered), 500)
    plt.hist(filtered, bins=100, density=True, alpha=0.6, edgecolor='k', label='Observed')

    best_fit = None
    best_ks = (None, 0)

    print("\n Distribution fitting results (Nose–Hoover–Langevin)")
    print("-" * 50)

    for name, dist in distributions.items():
        try:
            if name in ['gamma', 'expon']:
                params = dist.fit(filtered, floc=0)
            else:
                params = dist.fit(filtered)

            ks_stat, pval = stats.kstest(filtered, dist.name, args=params)
            pdf = dist.pdf(x, *params)
            plt.plot(x, pdf, lw=2, label=f'{name} (p={pval:.3f})')

            print(f"{name:<10s} KS={ks_stat:.4f}, p={pval:.4f}, params={np.round(params, 3)}")

            if pval > best_ks[1]:
                best_fit = name
                best_ks = (ks_stat, pval)

        except Exception as e:
            print(f"{name:<10s} Fitting failed: {e}")

    plt.xlabel("Steps to form 3 clusters")
    plt.ylabel("Probability Density")
    plt.title("Cluster Formation Time Distribution Fit (Nose–Hoover–Langevin)")
    plt.legend()
    plt.grid(True)
    plt.show()

    print(f"\n Best-fitting distribution: {best_fit} (p = {best_ks[1]:.3f})")
    return best_fit


In [ ]:
analyze_cluster_times("times_nosehoover_langevin.npy")

Langevin

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from scipy.stats import gamma
import sys

sys.path.append("../build")
from IPSModule import LangevinSystem, IPS_Simulator_Langevin

# -----------------------------
# Simulation parameters
# -----------------------------
gamma_ = 1.0
temperature = 0.5
dt = 0.0005
rad = 30.0
n_particles = 30
epsilon = 1.0
sigma = 1.0

# -----------------------------
# Initialize particle positions
# -----------------------------
def generate_random_particles(n_particles=30, box_radius=20.0):
    np.random.seed(None)
    while True:
        positions = np.random.uniform(-box_radius, box_radius, size=(n_particles, 2))
        if np.all(np.linalg.norm(positions, axis=1) <= box_radius):
            return positions

# -----------------------------
# Count clusters using DBSCAN
# -----------------------------
def count_clusters(positions, eps=1.7, min_samples=3):
    if np.isnan(positions).any():
        return 0
    db = DBSCAN(eps=eps, min_samples=min_samples)
    labels = db.fit_predict(positions)
    return len(set(labels)) - (1 if -1 in labels else 0)

# -----------------------------
# Single simulation: return the step when 3 clusters form
# -----------------------------
def simulate_until_five_clusters(max_steps=1500000, check_interval=500, eps=1.7, min_samples=3):
    init_positions = generate_random_particles(n_particles)
    system = LangevinSystem(n_particles, gamma_, temperature)

    for i in range(n_particles):
        for d in range(2):
            system.get_positions()[d][i] = init_positions[i][d]
            system.get_velocities()[d][i] = 0.0

    sim = IPS_Simulator_Langevin(system)
    sim.init(
        {"type": "LennardJones", "eps": epsilon, "sigma": sigma},
        {"type": "Radial", "rad": rad}
    )

    for step in range(max_steps):
        sim.integrate(dt)
        if step % check_interval == 0:
            pos = np.array(system.get_positions()).T
            n_clusters = count_clusters(pos, eps=eps, min_samples=min_samples)
            if n_clusters >= 3:
                return step
    return max_steps

# -----------------------------
# Run multiple simulations
# -----------------------------
def repeat_simulations(n_runs=500):
    times = []
    for i in range(n_runs):
        print(f"Run {i+1}/{n_runs}...")
        t = simulate_until_five_clusters()
        times.append(t)
        print(f"  Formed 3 clusters at step {t}")
    return times

# -----------------------------
# Plot cluster formation time histogram
# -----------------------------
def plot_formation_time_distribution(times, label="Langevin"):
    filtered = np.array([t for t in times if t < 1500000])
    counts, bins, _ = plt.hist(filtered, bins=30, edgecolor='k', alpha=0.6, label='Observed')

    # Optional: Fit gamma distribution
    # shape, loc, scale = gamma.fit(filtered, floc=0)
    # x = np.linspace(min(filtered), max(filtered), 500)
    # pdf = gamma.pdf(x, shape, loc=loc, scale=scale) * len(filtered) * (bins[1] - bins[0])
    # plt.plot(x, pdf, 'r-', lw=2, label=f'Gamma fit\nshape={shape:.2f}, scale={scale:.2f}')

    plt.xlabel("Steps to form 3 clusters")
    plt.ylabel("Frequency")
    plt.title(f"Cluster Formation Time Histogram ({label})")
    plt.legend()
    plt.show()

    print(f"Average steps: {np.mean(filtered):.2f} | Median: {np.median(filtered)}")

# Run simulation and plot results
times = repeat_simulations(n_runs=500)
np.save("times_langevin.npy", np.array(times))
plot_formation_time_distribution(times)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

def analyze_cluster_times_langevin(filename, max_steps=1_500_000):
    times = np.load(filename)
    filtered = times[times < max_steps]

    print(f"Loaded {len(times)} samples, with {len(times) - len(filtered)} unclustered samples filtered out.")

    distributions = {
        # 'gamma': stats.gamma,
        # 'weibull': stats.weibull_min,
        'lognorm': stats.lognorm,
        # 'expon': stats.expon
    }

    x = np.linspace(min(filtered), max(filtered), 500)
    plt.hist(filtered, bins=100, density=True, alpha=0.6, edgecolor='k', label='Observed')

    best_fit = None
    best_ks = (None, 0)

    print("\nDistribution fitting results (Langevin system)")
    print("-" * 50)

    for name, dist in distributions.items():
        try:
            if name in ['gamma', 'expon']:
                params = dist.fit(filtered, floc=0)
            else:
                params = dist.fit(filtered)

            ks_stat, pval = stats.kstest(filtered, dist.name, args=params)
            pdf = dist.pdf(x, *params)
            plt.plot(x, pdf, lw=2, label=f'{name} (p={pval:.3f})')

            print(f"{name:<10s} KS={ks_stat:.4f}, p={pval:.4f}, params={np.round(params, 3)}")

            if pval > best_ks[1]:
                best_fit = name
                best_ks = (ks_stat, pval)

        except Exception as e:
            print(f"{name:<10s} Fitting failed: {e}")

    plt.xlabel("Steps to form 3 clusters")
    plt.ylabel("Probability Density")
    plt.title("Cluster Formation Time Distribution Fit (Langevin System)")
    plt.legend()
    plt.grid(True)
    plt.show()

    print(f"\nBest-fitting distribution: {best_fit} (p = {best_ks[1]:.3f})")
    return best_fit


In [ ]:
analyze_cluster_times_langevin("times_langevin.npy")